In [34]:
import cv2
import os
import numpy as np
import matplotlib.pyplot as plt

face_cascade=cv2.CascadeClassifier("C:\\Users\\Naman\\Desktop\\projects\\haarcascade_frontalface_default.xml")

# Loading the data in arrays

In [35]:
  #it gives a list of all the folders within the category
def load_and_set_data(directory , categories , face_cascade):
    data=[]
    for cat in categories:
        cat_path=directory+"\\"+cat
        for img in os.listdir(cat_path):
            try:
                img_array=cv2.imread(cat_path+"\\"+img)
                gray=cv2.cvtColor(img_array , cv2.COLOR_BGR2GRAY)
                faces=face_cascade.detectMultiScale(gray , 1.3 , 5)
                for (x , y , w , h) in faces:
                    roi_image=gray[y:y+h , x:x+w]
                    new_roi_image=cv2.resize(roi_image , (100 , 100))
                    data.append([new_roi_image , cat])
            except Exception as e:
                pass
    return data
        
#now we have our data and the corresponding target
#all the images are gray scale

In [36]:
directory="C:\\Users\\Naman\\Desktop\\projects\\Face-Mask-Detection-master\\dataset"  #location of images
categories=os.listdir(directory)   #different categories of images
data=load_and_set_data(directory , categories , face_cascade)  #loading the data


In [37]:
import random
random.shuffle(data)   #shuffling the data

In [38]:
data=np.array(data)

In [39]:
data.shape

(878, 2)

In [40]:
len(data)

878

# Developing the model

In [41]:
from keras.layers import Conv2D , MaxPooling2D , Flatten , Dense   #importing the different layers
from keras.models import Sequential    #importing the model

In [42]:
def create_model():   #developing the model by adding different layers
    model=Sequential()
    model.add(Conv2D(100, (2, 2), strides=(1 , 1)  , activation='relu', padding="same" , input_shape=(100, 100, 1)))
    model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    model.add(Dense(100, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer="adam", loss="binary_crossentropy", metrics=['accuracy'])
    
    return model

In [43]:
input_=data[: , 0]
target=data[: , 1]

In [44]:
for i in range(len(target)):  #converting the label into binary codes
    if target[i]=="with_mask":
        target[i]=1
    else:
        target[i]=0
target=target.astype(np.float32)

In [45]:
from  sklearn.model_selection import train_test_split   

In [46]:
X_train , X_test , Y_train , Y_test=train_test_split(input_ , target , random_state=1)   #splitting the data

In [47]:
X_train_new=[]
for i in range(len(X_train)):
    X_train_new.append(np.reshape(X_train[i] , (100 , 100 , 1)))   #resizing the input and storing in  the array
    
X_test_new=[]
for i in range(len(X_test)):
    X_test_new.append(np.reshape(X_test[i] , (100 , 100 , 1)))     #resizing the input and storing in the array
    
X_train_new=np.array(X_train_new).astype(np.float32)
X_test_new=np.array(X_test_new).astype(np.float32)



In [48]:
mo=create_model()   #creating the model

In [49]:
mo.fit(X_train_new , Y_train , epochs=5 ,batch_size=10, validation_data=(X_test_new , Y_test))  #fitting the model

Train on 658 samples, validate on 220 samples
Epoch 1/5
658/658 [==============================] - 34s 52ms/step - loss: 154.6841 - accuracy: 0.8480 - val_loss: 79.1342 - val_accuracy: 0.8818
Epoch 2/5
658/658 [==============================] - 33s 50ms/step - loss: 19.6952 - accuracy: 0.9164 - val_loss: 15.5173 - val_accuracy: 0.9000
Epoch 3/5
658/658 [==============================] - 35s 53ms/step - loss: 3.3705 - accuracy: 0.9559 - val_loss: 9.6601 - val_accuracy: 0.9045
Epoch 4/5
658/658 [==============================] - 33s 50ms/step - loss: 2.1813 - accuracy: 0.9590 - val_loss: 9.1817 - val_accuracy: 0.8500
Epoch 5/5
658/658 [==============================] - 33s 50ms/step - loss: 1.6744 - accuracy: 0.9666 - val_loss: 28.7967 - val_accuracy: 0.8909


In [50]:
predictions=mo.predict(X_test_new)  #getting the predictions on testing data

In [51]:
real_predictions=[]     #converting the predictions from probabilities to labels
for i in range(len(predictions)):
    if predictions[i][0]>0.5:
        real_predictions.append(1)
    else:
        real_predictions.append(0)
        

In [52]:
(real_predictions==Y_test).sum()    #checking the number of correct predictions

196

In [53]:
#feeding data through webcam
def webcam_input():
    font=cv2.FONT_HERSHEY_SIMPLEX  #giving the font through
    cap=cv2.VideoCapture(0)     #creating an object for video capture
    while True:
        ret , image=cap.read()
        gray=cv2.cvtColor(image , cv2.COLOR_BGR2GRAY)   #converting the image to gray scale for easier computation
        
        faces=face_cascade.detectMultiScale(gray , 1.3 , 5)    #function used to detect faces
        for (x , y , w , h) in faces:
            roi_image=gray[y:y+h , x:x+w]      #gray image with face
            roi_image=cv2.resize(roi_image , (100 , 100))  #resizing it
            new_image=np.resize(roi_image , (1 , 100 , 100 , 1))
            ans=mo.predict(new_image)     #getting the prediction whether the person is wearing mask or not
            if ans[0]==1:
                cv2.rectangle(image , (x , y) , (x+w , y+h) , (0 , 0 , 255) , 2) #making the rectangle aroud the face
                lb="with_mask"     #getting the label 
            else:
                cv2.rectangle(image , (x , y) , (x+w , y+h) , (0 , 255 , 0) , 2)  #making the rectangle around the face
                lb="without_mask"   #getting the label
            cv2.putText(image ,lb , (x+w//2 , y+h) , font , 1 , (255 , 0 , 0) , 2 , cv2.LINE_AA)  #putting the label
        cv2.imshow("ans" , image)    #showing the output on screen
        if cv2.waitKey(1) & 0xFF==ord('q'):   #using q as key to quit
            break
    cap.release()
    cv2.destroyAllWindows()
      
            

In [54]:
webcam_input()